# Plot spatial properties

In this notebook we provide code to show the spatial distribution of extracted properties of cells using Napri

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%gui qt

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
matplotlib.rc("figure", figsize=(10,5))

import seaborn as sns

import pathlib

import delta
import napari


## Set paths

Set the paths to the position file and dataframe generated in notebooks 1 and 2

In [2]:
proj_dir = pathlib.Path(pathlib.Path.home(), 'I2ICourse', 'Project2B')
pos_file = proj_dir / 'ProcessedData' / 'Position000000.pkl'
df_file = proj_dir / 'ProcessedData' / 'Position000000_dataframe.csv'


## Define function

Below we define the function to create a spatial map of a property of choice

In [3]:
def plot_spatial_map(pos, df, property, frame=-1):
   #pos: delta position object
   #df: pandas data frame of Delta output
   #property: key of cell property contained in lineage object
   #frame: frame to show, if not specified last one is chosen
   
   #create color map where Nan is shown as black  
   colMap = cm.get_cmap("viridis").copy()
   colMap.set_bad(color='black')
   
   #get frame
   frame = len(pos.rois[0].label_stack)-1 if frame==-1 else frame

   # get label image:
   labels = pos.rois[0].label_stack[frame]
   spatial_map = np.full(labels.shape, np.nan)
   
   # Go over cells in selected frame:
   for cnb in np.unique(labels):
      if cnb==0: continue
      #assign cells mask area the phenotype of choice
      spatial_map[labels==cnb] = df.loc[(df['frames']==frame) & (df['id_seg']==cnb-1),property].item()
   
   return spatial_map

## Load Data
We load the data from the last notebook;

In [4]:
#postion object
pos = delta.pipeline.Position(None,None,None)
pos.load(pos_file)
df = pd.read_csv(df_file)

labels = pos.rois[0].label_stack
imstack = np.stack(pos.rois[0].img_stack, axis=0) #load image data from position file
labelstack = np.stack(labels, axis=0) #load label data from position file


## Plot data

you can set `property_to_show` to any column name in the dataframe

In [5]:
property_to_show = 'id_colony'
#loop over all frames
props = [plot_spatial_map(pos, df, property_to_show, frame=fr) for fr in range(len(labels))]
propstack = np.stack(props) #load label data from position file

Now we create a Napari window adding the property to show

In [6]:
viewer = napari.view_image(imstack)
viewer.add_labels(labelstack)
viewer.add_image(propstack)
viewer.show()